In [ ]:
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
)
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, Markdown, display
import datetime

In [ ]:
samples = [
      {
        "question": "How can residents report unplowed roads?", 
        "response": "Residents can report unplowed roads using the 'Report an Issue' feature in the SnowLine app or by calling the ADS hotline.",
        "context": "If a road remains unplowed past its scheduled window, residents are encouraged to use the 'Report an Issue' button in the SnowLine mobile app or call the ADS hotline to submit a service request."
    },
    {
        "question": "What is the usual snow plow schedule for urban areas?", 
        "response": "In urban areas, snow plows typically operate on a 4-hour rotation during active snowfall.",
        "context": "ADS maintains a 4-hour rotation for snow plowing in urban zones during active snow conditions. Priority routes such as emergency access roads are cleared first."
    },
    {
        "question": "Does ADS provide updates on school closures?", 
        "response": "ADS does not directly manage school closures, but the SnowLine app includes integrated updates from local school districts.",
        "context": "While ADS doesn't control school operations, the SnowLine app offers real-time updates from school districts across Alaska, including closures and delays due to weather."
    },
    {
        "question": "Is the SnowLine app available on both Android and iOS?", 
        "response": "Yes, the SnowLine app is available for free on both the Apple App Store and Google Play Store.",
        "context": "The ADS SnowLine mobile app is available for download on iOS and Android platforms. It provides live road updates, service request tracking, and weather alerts."
    },
    {
        "question": "What is the ADS policy on overnight street parking during snowstorms?", 
        "response": "During active snow events, overnight street parking is prohibited on primary routes to allow safe plow operations.",
        "context": "ADS enforces a no-parking rule on priority and emergency roads during snowstorms. Violators may be ticketed or towed to ensure uninterrupted plow service."
    }
]

## Initalization of client

In [ ]:
client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-02-8fc93094313c",
      location="global",
  )

## System Prompt

In [ ]:
data = [
    {  
        "Answer the question: " + item['question'] + " Context: " + it['context']"
        "response": item["post"],
    } for item in samples
]

eval_dataset = pd.DataFrame(data)

In [ ]:
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"ads-chatbot-{run_ts}"
)

In [ ]:
prompt_template = (
    "Instruction: {instruction}. Prompt: {context}. Post: {response}"
)
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"generate-announcemet-post-{run_ts}"
)
evaluation_results = []
evaluation_results.append(result)

Asking the bot for product recommendations based on preferences.

In [ ]:
from vertexai.preview.evaluation import notebook_utils
notebook_utils.display_eval_result(eval_result=result)